## ChatBot - MedBot

- Problem Statement: Develop a ChatBot for Medical queries of Patient

In [1]:
# import Libraries
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import string

import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

from warnings import filterwarnings
filterwarnings("ignore")

In [2]:
# Loading intents data 
words=[]
classes = []
documents = []
ignore_words = ['?', '!',',','.']
data_file = open('intents.json').read()
intents = json.loads(data_file)

In [3]:
intents['intents']

[{'tag': 'greetings',
  'patterns': ['hello',
   'hey',
   'hi',
   'good day',
   'greetings',
   "what's up?",
   'how is it going'],
  'responses': ['hello', 'hey!', 'what can i do for you?']},
 {'tag': 'goodbye',
  'patterns': ['cya',
   'see you later',
   'goodbye',
   'have a good day',
   'bye',
   'cao',
   'see ya'],
  'responses': ['have a nice day', 'goodbye']},
 {'tag': 'age',
  'patterns': ['how old',
   'how old are you?',
   'what is your age',
   'how old are you',
   'age?'],
  'responses': ['I get reborn after every compilation',
   'hey!',
   'my owners are averagely 20 years!']},
 {'tag': 'name',
  'patterns': ['what is your name',
   'what should i call you',
   "what's your name?",
   'who are you?',
   'can you tell me your name'],
  'responses': ['you can call me Medbot!',
   'i am Medbot!',
   'i am Medbot your medical assistant']},
 {'tag': 'common cold symptoms',
  'patterns': ['Runny or stuffy nose',
   'Sore throat',
   'Cough',
   'Congestion',
   'Slight

### Data Cleaning

In [4]:
# Extracting list of all words and documents - combination of sentence pattern and tags
for intent in intents['intents']:
    for pattern in intent['patterns']:

        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
classes

['greetings',
 'goodbye',
 'age',
 'name',
 'common cold symptoms',
 'fever symptoms',
 'Diabetes symptoms',
 'Depression symptoms',
 'Asthma symptoms',
 'common cold prevention',
 'fever prevention',
 'diabetes prevention',
 'depression prevention',
 'asthma prevention',
 'Consultation']

### Data Modelling

In [6]:
# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)


84 documents
15 classes ['Asthma symptoms', 'Consultation', 'Depression symptoms', 'Diabetes symptoms', 'age', 'asthma prevention', 'common cold prevention', 'common cold symptoms', 'depression prevention', 'diabetes prevention', 'fever prevention', 'fever symptoms', 'goodbye', 'greetings', 'name']
134 unique lemmatized words ["'s", '(', ')', 'a', 'ache', 'age', 'am', 'and', 'anxiety', 'any', 'appetite', 'are', 'asthma', 'at', 'available', 'blurry', 'body', 'breath', 'buy', 'bye', 'call', 'can', 'cao', 'change', 'chest', 'chill', 'cold', 'common', 'congestion', 'consultation', 'contact', 'cough', 'coughing', 'cya', 'day', 'death', 'dehydration', 'depression', 'diabetes', 'difficulty', 'doctor', 'drink', 'eat', 'extreme', 'fatigue', 'feeling', 'fever', 'flu', 'for', 'frequent', 'from', 'general', 'generally', 'getting', 'give', 'going', 'good', 'goodbye', 'greeting', 'have', 'headache', 'hello', 'help', 'hey', 'hi', 'hopeless', 'how', 'hunger', 'i', 'if', 'in', 'increased', 'interest', 

In [7]:
len(words)

134

#### Train and Test Split

In [8]:
df = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    df.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(df)
df = np.array(df)
# create train and test lists. X - patterns, Y - intents
X_train, X_test, y_train, y_test = train_test_split(list(df[:,0]),list(df[:,1]),test_size=.3, random_state=1)
print("Training data created")


Training data created


### Model Development - CNN based model

In [9]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(X_train[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
model.fit(np.array(X_train), np.array(y_train), epochs=200, batch_size=3, verbose=1)


print("model created")

Epoch 1/200
20/20 [==============================] - 1s 3ms/step - loss: 2.7071 - accuracy: 0.0141
Epoch 2/200
20/20 [==============================] - 0s 3ms/step - loss: 2.7093 - accuracy: 0.0735
Epoch 3/200
20/20 [==============================] - 0s 2ms/step - loss: 2.7184 - accuracy: 0.0339
Epoch 4/200
20/20 [==============================] - 0s 2ms/step - loss: 2.7400 - accuracy: 0.0507
Epoch 5/200
20/20 [==============================] - 0s 2ms/step - loss: 2.6098 - accuracy: 0.0468
Epoch 6/200
20/20 [==============================] - 0s 2ms/step - loss: 2.5704 - accuracy: 0.2623
Epoch 7/200
20/20 [==============================] - 0s 2ms/step - loss: 2.5780 - accuracy: 0.1169
Epoch 8/200
20/20 [==============================] - 0s 2ms/step - loss: 2.6404 - accuracy: 0.0892
Epoch 9/200
20/20 [==============================] - 0s 3ms/step - loss: 2.5801 - accuracy: 0.3017
Epoch 10/200
20/20 [==============================] - 0s 2ms/step - loss: 2.5653 - accuracy: 0.2243
Epoch 11/

20/20 [==============================] - 0s 2ms/step - loss: 1.4773 - accuracy: 0.5168
Epoch 83/200
20/20 [==============================] - 0s 2ms/step - loss: 1.3229 - accuracy: 0.6621
Epoch 84/200
20/20 [==============================] - 0s 2ms/step - loss: 1.3391 - accuracy: 0.7712
Epoch 85/200
20/20 [==============================] - 0s 2ms/step - loss: 1.4323 - accuracy: 0.5208
Epoch 86/200
20/20 [==============================] - 0s 2ms/step - loss: 1.3282 - accuracy: 0.6454
Epoch 87/200
20/20 [==============================] - 0s 2ms/step - loss: 1.4078 - accuracy: 0.5053
Epoch 88/200
20/20 [==============================] - 0s 2ms/step - loss: 1.4307 - accuracy: 0.6535
Epoch 89/200
20/20 [==============================] - 0s 2ms/step - loss: 1.3596 - accuracy: 0.6331
Epoch 90/200
20/20 [==============================] - 0s 3ms/step - loss: 1.2319 - accuracy: 0.7125
Epoch 91/200
20/20 [==============================] - 0s 2ms/step - loss: 1.3678 - accuracy: 0.5966
Epoch 92/200


20/20 [==============================] - 0s 2ms/step - loss: 0.6407 - accuracy: 0.8455
Epoch 164/200
20/20 [==============================] - 0s 2ms/step - loss: 0.7795 - accuracy: 0.7831
Epoch 165/200
20/20 [==============================] - 0s 2ms/step - loss: 0.6604 - accuracy: 0.7974
Epoch 166/200
20/20 [==============================] - 0s 2ms/step - loss: 0.6512 - accuracy: 0.8112
Epoch 167/200
20/20 [==============================] - 0s 2ms/step - loss: 0.5916 - accuracy: 0.8459
Epoch 168/200
20/20 [==============================] - 0s 2ms/step - loss: 0.6649 - accuracy: 0.7336
Epoch 169/200
20/20 [==============================] - 0s 2ms/step - loss: 0.7085 - accuracy: 0.8504
Epoch 170/200
20/20 [==============================] - 0s 2ms/step - loss: 0.5646 - accuracy: 0.8377
Epoch 171/200
20/20 [==============================] - 0s 2ms/step - loss: 0.5004 - accuracy: 0.8787
Epoch 172/200
20/20 [==============================] - 0s 2ms/step - loss: 0.5625 - accuracy: 0.8304
Epoc

In [23]:
# Train prediction
res = model.predict(np.array(X_train))

In [24]:
def Accuracy(actual,pred):
    
    

    X=[]
    for p in actual:
    
        X.extend([classes[i] for i,r in enumerate(p) if r==1])
    
    results=[]
    for p in pred:
    
        results.extend([classes[i] for i,r in enumerate(p) if r== max(p)])
    
    a =[1 if i==j else 0 for i,j in zip(X,results) ]
        
    acc=sum(a)*100/len(a)
    
    return(acc) 
        
    

In [25]:
# Accuracy of Train data 
Accuracy(y_train,res)

100.0

In [26]:
# Test Prediction
res = model.predict(np.array(X_test))

In [27]:
# Accuracy of Test data 
Accuracy(y_test,res)

26.923076923076923

In [28]:
# functions for chat message for Data cleaning, modelling, labeling, and prediction and chat response

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def chatbot_response(msg):
    ints = predict_class(msg, model)
    if ints==[]:
        res='How can I help you?'
    else:
        
        
        tag = ints[0]['intent']
        list_of_intents = intents['intents']
        for i in list_of_intents:
            if(i['tag']== tag):
                res = random.choice(i['responses'])
            
                break
            
    return res


In [29]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
    
        res = chatbot_response(msg) 
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 



In [30]:
base = Tk()
base.title("Medbot")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview)
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )


In [31]:

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)



In [32]:
base.mainloop()

Conclusion : Chat designed for 15 claases using small samples of data, currently the model is overfitting due to less amount of 
             data and class Imbalance.